<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/English/English_Crossval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install focal-loss
#!pip install keras-tcn==2.8.3
!pip install keras-tcn
!pip install tensorflow==2.1.0
#!pip install git+https://github.com/philipperemy/keras-tcn.git


  Using cached https://files.pythonhosted.org/packages/85/d4/c0cd1057b331bc38b65478302114194bd8e1b9c2bbc06e300935c0e93d90/tensorflow-2.1.0-cp36-cp36m-manylinux2010_x86_64.whl


In [0]:
import tensorflow as tf
# tf.enable_eager_execution()


In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Import lib

In [38]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential,Model

from tensorflow.keras.layers import Embedding,GRU,LSTM,Dense,Dropout,Bidirectional,BatchNormalization,GlobalMaxPooling1D,Flatten, GlobalAveragePooling1D, MaxPooling1D,SpatialDropout1D,Input,Activation,concatenate,Conv1D,multiply
from tensorflow.keras.optimizers import RMSprop,Adam,Adadelta
from tensorflow.keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
# import keras
import collections
import numpy as np
from focal_loss import BinaryFocalLoss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from tensorflow.keras import backend as K
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
from tensorflow.keras.layers import Layer,Lambda,InputSpec
from tensorflow.keras import initializers
from tensorflow.keras import backend as K
# import tensorflow as tf
# tf.logging.set_verbosity(tf.logging.ERROR)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Load data

In [5]:
train=pd.read_csv( '/content/drive/My Drive/dataset/OLID/data/olid-training-v1.0.tsv',sep="\t")
train.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [6]:
val=pd.read_csv('/content/drive/My Drive/dataset/OLID/data/testset-levela.tsv',sep="\t")
val_labels=pd.read_csv('/content/drive/My Drive/dataset/OLID/data/labels-levela.csv',sep=",",header=None)
val_labels.columns=['idx','subtask_a']
#val_labels.head()
val=pd.concat([val,val_labels],axis=1)
val.head()

,id,tweet,idx,subtask_a
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,15923,OFF
1,27014,"#ConstitutionDay is revered by Conservatives, ...",27014,NOT
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,30530,NOT
3,13876,#Watching #Boomer getting the news that she is...,13876,NOT
4,60133,#NoPasaran: Unity demo to oppose the far-right...,60133,OFF


In [0]:
train_labels=train['subtask_a']
train=train['tweet']


val_labels=val['subtask_a']
val=val['tweet']


In [0]:
train=pd.concat([train,val])
labels=pd.concat([train_labels,val_labels])

In [9]:
test=pd.read_csv( '/content/drive/My Drive/dataset/OffenseEval2020/data/English/test_a_tweets.tsv',sep="\t")
test.head()

,id,tweet
0,A0,¿Who the fuck is Yoru?
1,A1,@USER @USER He's an evil law breaker that shou...
2,A2,Now hiring for 49 #Labor job opportunities in ...
3,A3,"#NerkondaPaarvai' - As Of Now, There Is NO Ear..."
4,A4,"@USER Hahahaha I wish...but a week is good, I’..."


In [0]:
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop = stopwords.words('english')


In [11]:
train.head()

0    @USER She should ask a few native Americans wh...
1    @USER @USER Go home you’re drunk!!! @USER #MAG...
2    Amazon is investigating Chinese employees who ...
3    @USER Someone should'veTaken" this piece of sh...
4    @USER @USER Obama wanted liberals &amp; illega...
Name: tweet, dtype: object

In [12]:

train=train.str.replace('\d+', '')
train=train.str.replace('@USER', "") 
train=train.str.replace('URL', '')
train= train.str.lower()
#train = train.str.replace('[^\w\s]','')
#train = train.apply(lambda x : [lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(x)])
#train = train.apply(lambda x: [item for item in x if item not in stop])
#train = train.apply(lambda x : " ".join(x))
train.head()

0     she should ask a few native americans what th...
1          go home you’re drunk!!!  #maga #trump 👊🇺🇸👊 
2    amazon is investigating chinese employees who ...
3     someone should'vetaken" this piece of shit to...
4      obama wanted liberals &amp; illegals to move...
Name: tweet, dtype: object

In [13]:

test['tweet']=test['tweet'].str.replace('\d+', '')
test['tweet']=test['tweet'].str.replace('@USER', "") 
test['tweet']=test['tweet'].str.replace('URL', '')
test["tweet"]= test["tweet"].str.lower()
# test["tweet"] = test["tweet"].str.replace('[^\w\s]','')
# test["tweet"] = test["tweet"].apply(lambda x : [lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(x)])
# test["tweet"] = test["tweet"].apply(lambda x: [item for item in x if item not in stop])
# test["tweet"] = test["tweet"].apply(lambda x : " ".join(x))
test.head()

,id,tweet
0,A0,¿who the fuck is yoru?
1,A1,he's an evil law breaker that should be in p...
2,A2,now hiring for #labor job opportunities in #m...
3,A3,"#nerkondapaarvai' - as of now, there is no ear..."
4,A4,"hahahaha i wish...but a week is good, i’m ext..."


#Label Encoding

In [0]:
test=test['tweet']


In [15]:
train.head()

0     she should ask a few native americans what th...
1          go home you’re drunk!!!  #maga #trump 👊🇺🇸👊 
2    amazon is investigating chinese employees who ...
3     someone should'vetaken" this piece of shit to...
4      obama wanted liberals &amp; illegals to move...
Name: tweet, dtype: object

In [16]:
le=LabelEncoder()
labels=le.fit_transform(labels)
print(len(labels))

14100


In [0]:
train_bert=train.copy()
test_bert=test.copy()

# Common Parameters

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

## Cylic learning rate

In [19]:
# https://www.kaggle.com/hireme/fun-api-keras-f1-metric-cyclical-learning-rate/code
from keras.callbacks import Callback
class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

Using TensorFlow backend.


In [0]:
clr = CyclicLR(base_lr=0.001, max_lr=0.005,
                        step_size=4., mode='exp_range',
                        gamma=0.99994)

# Tokenize data

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 150

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(pd.concat([train,test]))
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len,padding='post')

In [22]:
len(sequences_matrix),len(labels)

(14100, 14100)

In [23]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

10001


In [0]:
test_sequences = tokenize.texts_to_sequences(test)
X_test_sequences = sequence.pad_sequences(test_sequences,maxlen=max_len,padding='post')

In [25]:
len(X_test_sequences)

3887

# Embeddings

In [0]:
embedding_path1 = "drive/My Drive/dataset/OLID/wiki-news-300d-1M.vec"
embedding_path2 = "drive/My Drive/dataset/OLID/glove/glove.840B.300d.txt"
embed_size = 300

In [0]:
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
fasttext=build_matrix(embedding_path1, word_index)
glove_emb=build_matrix(embedding_path2, word_index)

In [29]:
embeddings=np.mean((fasttext,glove_emb),axis=0)
embeddings.shape

(10001, 300)

In [0]:
#np.save('/content/drive/My Drive/dataset/OffenseEval2020/data/English/en_embeddings.npy',embeddings)

# Base Models

In [0]:
train.shape,labels.shape

## Count Vectorizer

In [0]:

def logistic_param_selection(X, y, func,nfolds):
    C= [7,8,9,10,12,15,20,25]
    param_grid = {'C': C}
    grid_search = GridSearchCV(make_pipeline(func, LogisticRegression(solver='lbfgs',max_iter=500,class_weight=class_weights)),
                    param_grid={'logisticregression__C':C}, cv=nfolds,scoring='f1_macro')
    grid_search.fit(X, y)
    return grid_search.best_score_,grid_search.best_params_



In [0]:
cv_score,c=logistic_param_selection(train,labels,CountVectorizer(),5)
cv_score

In [0]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit(train)
X_train_cv = count_vectorizer.transform(train)
X_test_cv=  count_vectorizer.transform(test)

In [0]:
cv_classifier = LogisticRegression(solver='lbfgs',C=c['logisticregression__C'],max_iter=500,class_weight=class_weights)
cv_classifier.fit(X_train_cv, labels)

In [0]:
cv_test=cv_classifier.predict_proba(X_test_cv)[:,1]

## TF IDF word vectorizer

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 4),
    max_features=10000)
word_vectorizer.fit(pd.concat([train, test]))
train_word_features = word_vectorizer.transform(train)
test_word_features = word_vectorizer.transform(test)

In [0]:
tfw_score,c=logistic_param_selection(train,labels,word_vectorizer,5)
tfw_score

In [0]:
tfw_classifier = LogisticRegression(solver='lbfgs',C=c['logisticregression__C'],max_iter=500)
tfw_classifier.fit(train_word_features, labels)


In [0]:
tfw_test = tfw_classifier.predict_proba(test_word_features)[:,1]

## TF IDF char vectorizer

In [0]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 6),
    max_features=30000)
char_vectorizer.fit(train)
train_char_features = char_vectorizer.transform(train)
test_char_features = char_vectorizer.transform(test)

In [0]:
tfc_score,c=logistic_param_selection(train,labels,char_vectorizer,5)
tfc_score

In [0]:
tfc_classifier = LogisticRegression(solver='lbfgs',C=c['logisticregression__C'],max_iter=500)
tfc_classifier.fit(train_char_features, labels)


In [0]:
tfc_test = tfc_classifier.predict_proba(test_char_features)[:,1]

# Models

##GRU

In [0]:
def model_gru():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.1)(x)
  x = Bidirectional(LSTM(50, return_sequences=True))(x)
  x, x_h, x_c = Bidirectional(GRU(50, activation= 'tanh',recurrent_activation ='sigmoid',recurrent_dropout=0,
                      unroll =False,use_bias =True,reset_after=True,return_sequences = True, return_state = True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  conc = concatenate([avg_pool, x_h, max_pool])
  outp = Dense(1, activation="sigmoid")(conc)    
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
def model_gru1():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.2)(x)
  x=Bidirectional(GRU(150,activation= 'tanh',recurrent_activation ='sigmoid',recurrent_dropout=0,
                      unroll =False,use_bias =True,reset_after=True,return_sequences = True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  conc = concatenate([avg_pool, max_pool])


  x=Dropout(0.1)(conc)
  out=Dense(64, activation="relu")(x)
  out=Dense(32, activation="relu")(x)

  outp=Dense(1, activation="sigmoid")(x)   
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model

## GRU attention

In [0]:
#https://github.com/zake7749/DeepToxic


class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weight = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [0]:
def model_gru_attn():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.1)(x)
  x = Bidirectional(LSTM(50, activation= 'tanh',recurrent_activation ='sigmoid',recurrent_dropout=0,
                      unroll =False,use_bias =True,return_sequences = True))(x)
  x = Bidirectional(GRU(50, activation= 'tanh',recurrent_activation ='sigmoid',recurrent_dropout=0,
                      unroll =False,use_bias =True,reset_after=True,return_sequences = True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  last = Lambda(lambda t: t[:, -1])(x)
  attn = AttentionWeightedAverage()(x)
  conc = concatenate([avg_pool,  max_pool,last,attn])
  outp = Dense(1, activation="sigmoid")(conc)    
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model

##TCN

In [0]:
from tcn import TCN

def wave_net_activation(x):
    # type: (Layer) -> Layer
    """This method defines the activation used for WaveNet
    described in https://deepmind.com/blog/wavenet-generative-model-raw-audio/
    Args:
        x: The layer we want to apply the activation to
    Returns:
        A new layer with the wavenet activation applied
    """
    tanh_out = Activation('tanh')(x)
    sigm_out = Activation('sigmoid')(x)
    return multiply([tanh_out, sigm_out])

def model_tcn(embedding_matrix):
    
    inp = Input(shape=(max_len,))
    x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
    x = SpatialDropout1D(0.1)(x)
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True ,name = 'tnc1')(x)
    x = wave_net_activation(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, name = 'tnc2')(x)
    x = wave_net_activation(x)
    #x = TCN(32,dilations = [1, 2, 4], return_sequences=True, activation = 'wavenet',name = 'tnc3')(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dense(32, activation="relu")(conc)

    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
    
    return model

##KIM

In [0]:
conv_filters=128
def model_kim():
  inp = Input(shape=(max_len,))
  emb = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  # Specify each convolution layer and their kernel siz i.e. n-grams 
  conv1_1 = Conv1D(filters=conv_filters, kernel_size=3)(emb)
  btch1_1 = BatchNormalization()(conv1_1)
  drp1_1  = Dropout(0.2)(btch1_1)
  actv1_1 = Activation('relu')(drp1_1)
  glmp1_1 = GlobalMaxPooling1D()(actv1_1)

  conv1_2 = Conv1D(filters=conv_filters, kernel_size=4)(emb)
  btch1_2 = BatchNormalization()(conv1_2)
  drp1_2  = Dropout(0.2)(btch1_2)
  actv1_2 = Activation('relu')(drp1_2)
  glmp1_2 = GlobalMaxPooling1D()(actv1_2)

  conv1_3 = Conv1D(filters=conv_filters, kernel_size=5)(emb)
  btch1_3 = BatchNormalization()(conv1_3)
  drp1_3  = Dropout(0.2)(btch1_3)
  actv1_3 = Activation('relu')(drp1_3)
  glmp1_3 = GlobalMaxPooling1D()(actv1_3)

  conv1_4 = Conv1D(filters=conv_filters, kernel_size=6)(emb)
  btch1_4 = BatchNormalization()(conv1_4)
  drp1_4  = Dropout(0.2)(btch1_4)
  actv1_4 = Activation('relu')(drp1_4)
  glmp1_4 = GlobalMaxPooling1D()(actv1_4)

  # Gather all convolution layers
  cnct = concatenate([glmp1_1, glmp1_2, glmp1_3, glmp1_4], axis=1)
  drp1 = Dropout(0.2)(cnct)

  dns1  = Dense(32, activation='relu')(drp1)
  btch1 = BatchNormalization()(dns1)
  drp2  = Dropout(0.2)(btch1)

  out = Dense(1, activation='sigmoid')(drp2)   
  model = Model(inputs=inp, outputs=out)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model


##RNN_CNN

In [0]:

def model_gru_cnn():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.5)(x)
  x = Bidirectional(LSTM(128, activation= 'tanh',recurrent_activation ='sigmoid',recurrent_dropout=0,
                      unroll =False,use_bias =True,return_sequences = True))(x)
  x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  x = concatenate([avg_pool, max_pool])
  x = Dense(10, activation = "relu")(x)

  outp=Dense(1, activation="sigmoid")(x)   
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model


# Train model

##GRU

In [49]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
gru_scores = []
gru_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  gru_model=model_gru()
  
  gru_model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = gru_model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  gru_test.append(gru_model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  gru_scores.append(f1)
  tf.keras.backend.clear_session()


Train on 11280 samples, validate on 2820 samples
Epoch 1/10
11280/11280 - 32s - loss: 0.1274 - accuracy: 0.7660 - val_loss: 0.1179 - val_accuracy: 0.7883
Epoch 2/10
11280/11280 - 26s - loss: 0.1146 - accuracy: 0.8005 - val_loss: 0.1163 - val_accuracy: 0.7936
Epoch 3/10
11280/11280 - 26s - loss: 0.1105 - accuracy: 0.8097 - val_loss: 0.1172 - val_accuracy: 0.7915
Epoch 4/10
11280/11280 - 26s - loss: 0.1074 - accuracy: 0.8163 - val_loss: 0.1199 - val_accuracy: 0.7830
Epoch 5/10
11280/11280 - 26s - loss: 0.0978 - accuracy: 0.8389 - val_loss: 0.1427 - val_accuracy: 0.7727
Epoch 6/10
11280/11280 - 26s - loss: 0.0950 - accuracy: 0.8424 - val_loss: 0.1355 - val_accuracy: 0.7858
Epoch 7/10
11280/11280 - 26s - loss: 0.0789 - accuracy: 0.8754 - val_loss: 0.1361 - val_accuracy: 0.7798
Epoch 8/10
11280/11280 - 26s - loss: 0.0721 - accuracy: 0.8876 - val_loss: 0.1363 - val_accuracy: 0.7851
Epoch 9/10
11280/11280 - 26s - loss: 0.0604 - accuracy: 0.9121 - val_loss: 0.1580 - val_accuracy: 0.7699
Epoch 

In [50]:
np.mean(gru_scores)

0.7348080576258096

##GRU 1

In [51]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
gru_scores1 = []
gru_test1=[]
for train, val in kfold.split(sequences_matrix, labels):
  gru_model=model_gru1()
  
  gru_model.fit(sequences_matrix[train], labels[train],batch_size=128,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]))
  y_pred = gru_model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  gru_test1.append(gru_model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  gru_scores1.append(f1)
  tf.keras.backend.clear_session()

Train on 11280 samples, validate on 2820 samples
Epoch 1/10
11280/11280 - 12s - loss: 0.1326 - accuracy: 0.7473 - val_loss: 0.1202 - val_accuracy: 0.7833
Epoch 2/10
11280/11280 - 9s - loss: 0.1168 - accuracy: 0.7925 - val_loss: 0.1182 - val_accuracy: 0.7865
Epoch 3/10
11280/11280 - 9s - loss: 0.1127 - accuracy: 0.8000 - val_loss: 0.1175 - val_accuracy: 0.7904
Epoch 4/10
11280/11280 - 9s - loss: 0.1094 - accuracy: 0.8094 - val_loss: 0.1178 - val_accuracy: 0.7876
Epoch 5/10
11280/11280 - 9s - loss: 0.1059 - accuracy: 0.8171 - val_loss: 0.1199 - val_accuracy: 0.7801
Epoch 6/10
11280/11280 - 9s - loss: 0.1028 - accuracy: 0.8258 - val_loss: 0.1229 - val_accuracy: 0.7894
Epoch 7/10
11280/11280 - 9s - loss: 0.0990 - accuracy: 0.8328 - val_loss: 0.1186 - val_accuracy: 0.7787
Epoch 8/10
11280/11280 - 9s - loss: 0.0932 - accuracy: 0.8463 - val_loss: 0.1205 - val_accuracy: 0.7887
Epoch 9/10
11280/11280 - 9s - loss: 0.0891 - accuracy: 0.8574 - val_loss: 0.1280 - val_accuracy: 0.7876
Epoch 10/10
11

In [52]:
np.mean(gru_scores1)

0.7556838389107827

## GRU attention

In [53]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
atten_scores = []
atten_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  attn_gru_model=model_gru_attn()
  
  attn_gru_model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = attn_gru_model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  atten_test.append(attn_gru_model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  atten_scores.append(f1)
  tf.keras.backend.clear_session()

Train on 11280 samples, validate on 2820 samples
Epoch 1/10
11280/11280 - 32s - loss: 0.1275 - accuracy: 0.7618 - val_loss: 0.1190 - val_accuracy: 0.7851
Epoch 2/10
11280/11280 - 26s - loss: 0.1147 - accuracy: 0.7999 - val_loss: 0.1169 - val_accuracy: 0.7901
Epoch 3/10
11280/11280 - 26s - loss: 0.1110 - accuracy: 0.8063 - val_loss: 0.1175 - val_accuracy: 0.7904
Epoch 4/10
11280/11280 - 26s - loss: 0.1084 - accuracy: 0.8139 - val_loss: 0.1174 - val_accuracy: 0.7883
Epoch 5/10
11280/11280 - 26s - loss: 0.1006 - accuracy: 0.8316 - val_loss: 0.1199 - val_accuracy: 0.7908
Epoch 6/10
11280/11280 - 26s - loss: 0.0943 - accuracy: 0.8453 - val_loss: 0.1222 - val_accuracy: 0.7872
Epoch 7/10
11280/11280 - 26s - loss: 0.0865 - accuracy: 0.8571 - val_loss: 0.1250 - val_accuracy: 0.7851
Epoch 8/10
11280/11280 - 26s - loss: 0.0740 - accuracy: 0.8855 - val_loss: 0.1356 - val_accuracy: 0.7833
Epoch 9/10
11280/11280 - 26s - loss: 0.0632 - accuracy: 0.9068 - val_loss: 0.1515 - val_accuracy: 0.7624
Epoch 

In [55]:
 np.mean(atten_scores)

0.7335868950649289

##TCN

In [56]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
tcn_scores = []
tcn_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  model=model_tcn(embeddings)
  
  model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  tcn_test.append(model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  tcn_scores.append(f1)
  tf.keras.backend.clear_session()

Train on 11280 samples, validate on 2820 samples
Epoch 1/10
11280/11280 - 11s - loss: 0.1545 - accuracy: 0.6800 - val_loss: 0.1330 - val_accuracy: 0.7599
Epoch 2/10
11280/11280 - 8s - loss: 0.1281 - accuracy: 0.7707 - val_loss: 0.1305 - val_accuracy: 0.7670
Epoch 3/10
11280/11280 - 8s - loss: 0.1224 - accuracy: 0.7887 - val_loss: 0.1343 - val_accuracy: 0.7652
Epoch 4/10
11280/11280 - 8s - loss: 0.1194 - accuracy: 0.7943 - val_loss: 0.1221 - val_accuracy: 0.7780
Epoch 5/10
11280/11280 - 8s - loss: 0.1166 - accuracy: 0.7939 - val_loss: 0.1213 - val_accuracy: 0.7738
Epoch 6/10
11280/11280 - 8s - loss: 0.1184 - accuracy: 0.7899 - val_loss: 0.1236 - val_accuracy: 0.7745
Epoch 7/10
11280/11280 - 8s - loss: 0.1147 - accuracy: 0.8027 - val_loss: 0.1244 - val_accuracy: 0.7908
Epoch 8/10
11280/11280 - 8s - loss: 0.1132 - accuracy: 0.8034 - val_loss: 0.1203 - val_accuracy: 0.7855
Epoch 9/10
11280/11280 - 8s - loss: 0.1099 - accuracy: 0.8105 - val_loss: 0.1209 - val_accuracy: 0.7922
Epoch 10/10
11

In [57]:
np.mean(tcn_scores)

0.7444826147601272

##KIM

In [58]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
kim_scores = []
kim_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  model=model_kim()
  
  model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  kim_test.append(model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  kim_scores.append(f1)
  tf.keras.backend.clear_session()

Train on 11280 samples, validate on 2820 samples
Epoch 1/10
11280/11280 - 9s - loss: 0.2076 - accuracy: 0.6854 - val_loss: 0.2119 - val_accuracy: 0.6709
Epoch 2/10
11280/11280 - 6s - loss: 0.1377 - accuracy: 0.7612 - val_loss: 0.1799 - val_accuracy: 0.6755
Epoch 3/10
11280/11280 - 6s - loss: 0.1196 - accuracy: 0.7947 - val_loss: 0.1384 - val_accuracy: 0.7418
Epoch 4/10
11280/11280 - 6s - loss: 0.1045 - accuracy: 0.8223 - val_loss: 0.1398 - val_accuracy: 0.7628
Epoch 5/10
11280/11280 - 6s - loss: 0.0975 - accuracy: 0.8365 - val_loss: 0.1281 - val_accuracy: 0.7809
Epoch 6/10
11280/11280 - 6s - loss: 0.0842 - accuracy: 0.8668 - val_loss: 0.1313 - val_accuracy: 0.7865
Epoch 7/10
11280/11280 - 6s - loss: 0.0737 - accuracy: 0.8818 - val_loss: 0.1363 - val_accuracy: 0.7610
Epoch 8/10
11280/11280 - 6s - loss: 0.0675 - accuracy: 0.8989 - val_loss: 0.1455 - val_accuracy: 0.7833
Epoch 9/10
11280/11280 - 6s - loss: 0.0568 - accuracy: 0.9152 - val_loss: 0.1499 - val_accuracy: 0.7773
Epoch 10/10
112

In [59]:
np.mean(kim_scores)

0.7245472865145907

##GRU_CNN

In [60]:
kfold = StratifiedKFold(n_splits=5, shuffle=False)
grucnn_scores = []
grucnn_test=[]
for train, val in kfold.split(sequences_matrix, labels):
  model=model_gru_cnn()
  
  model.fit(sequences_matrix[train], labels[train],batch_size=64,epochs=10,verbose=2,
            validation_data=(sequences_matrix[val],labels[val]),callbacks=[clr,])
  y_pred = model.predict(sequences_matrix[val], batch_size=64, verbose=1)

  grucnn_test.append(model.predict(X_test_sequences, batch_size=64, verbose=1).ravel())

  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[val], y_pred, average='macro')
  grucnn_scores.append(f1)
  tf.keras.backend.clear_session()

Train on 11280 samples, validate on 2820 samples
Epoch 1/10
11280/11280 - 16s - loss: 0.1353 - accuracy: 0.7413 - val_loss: 0.1228 - val_accuracy: 0.7755
Epoch 2/10
11280/11280 - 13s - loss: 0.1224 - accuracy: 0.7801 - val_loss: 0.1199 - val_accuracy: 0.7805
Epoch 3/10
11280/11280 - 13s - loss: 0.1200 - accuracy: 0.7873 - val_loss: 0.1175 - val_accuracy: 0.7929
Epoch 4/10
11280/11280 - 13s - loss: 0.1172 - accuracy: 0.7936 - val_loss: 0.1180 - val_accuracy: 0.7887
Epoch 5/10
11280/11280 - 13s - loss: 0.1158 - accuracy: 0.7955 - val_loss: 0.1171 - val_accuracy: 0.7879
Epoch 6/10
11280/11280 - 13s - loss: 0.1142 - accuracy: 0.7947 - val_loss: 0.1166 - val_accuracy: 0.7848
Epoch 7/10
11280/11280 - 13s - loss: 0.1108 - accuracy: 0.8050 - val_loss: 0.1184 - val_accuracy: 0.7901
Epoch 8/10
11280/11280 - 13s - loss: 0.1096 - accuracy: 0.8068 - val_loss: 0.1233 - val_accuracy: 0.7635
Epoch 9/10
11280/11280 - 13s - loss: 0.1060 - accuracy: 0.8153 - val_loss: 0.1261 - val_accuracy: 0.7851
Epoch 

In [61]:
np.mean(grucnn_scores)

0.7553129479407418

#**Result**

# Result

In [0]:
gru_scores=np.average(np.array(gru_scores))
atten_scores=np.average(np.array(atten_scores))
tcn_scores=np.average(np.array(tcn_scores))
kim_scores=np.average(np.array(kim_scores))

In [0]:
print('f1 score of count vec' ,cv_score)
print('f1 score of word tfidf' ,tfw_score)
print('f1 score of char tfidf' ,tfc_score)

print('f1 score of RNN' ,gru_scores)
print('f1 score of gated attention',atten_scores )
print('f1 score of tcn',tcn_scores)
print('f1 score of kim',kim_scores)


In [0]:
kim_test_avg=np.mean(np.array(kim_test),axis=0)
gru_test_avg=np.mean(np.array(gru_test),axis=0)
atten_test_avg=np.mean(np.array(atten_test),axis=0)
tcn_test_avg=np.mean(np.array(tcn_test),axis=0)


In [0]:
kim_test[0].shape,cv_test.shape,kim_test_avg.shape

In [0]:
print(cv_test[5:15].round())
print(tfw_test[5:15].round())
print(tfc_test[5:15].round())
print(gru_test_avg[5:15].round())
print(atten_test_avg[5:15].round())
print(kim_test_avg[5:15].round())
print(tcn_test_avg[5:15].round())


# Ensemble

we will choose only those, having f1 greater than 0.7. Once they are selected, we will decode labels, and keep label according to mod

In [0]:
wt=gru_scores+atten_scores+cv_score+tfw_score+tfc_score+tcn_scores+kim_scores
wt

In [0]:
#i am bit changing the distribution in order to give more weightage to ml than dl
y_test=gru_scores/wt * gru_test_avg + atten_scores/wt * atten_test_avg + cv_score/wt * cv_test + tfw_score/wt *tfw_test + tfc_score/wt *tfc_test +kim_scores/wt * kim_test_avg + tcn_scores/wt *tcn_test_avg

In [0]:
y_test[5:15].round()

In [0]:
def decode(y_test):
  y_test[y_test>0.5]=1
  y_test[y_test<0.5]=0
  y_test=y_test.astype('int16').ravel()

  y_test=le.inverse_transform(y_test)
  y_test=pd.DataFrame(y_test,columns=['label'])
  y_test=pd.concat([ids, y_test['label']], axis=1)
  return y_test

In [0]:
y_test[y_test>0.5]=1
y_test[y_test<0.5]=0
y_test=y_test.astype('int16').ravel()

y_test=pd.DataFrame(y_test,columns=['label'])
y_test=pd.concat([ids, y_test['label']], axis=1)
y_test.head()

In [0]:
y_test.head()

# Submit file

In [0]:
y_test.to_csv('/content/disaster.csv',index=False,header=None)


# Reset

In [0]:
# import keras
# keras.backend.clear_session()